In [1]:
import mysql.connector
from datetime import datetime
import mysql.connector
import pandas as pd
import warnings
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import time
import numpy as np 
import matplotlib.gridspec as gridspec
from matplotlib.widgets import Button
import matplotlib.dates as mdates
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from gym import spaces

In [7]:
def get_table_names(host, user, password, database):
    try:
        # Establish a database connection
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        # Create a cursor object
        cursor = conn.cursor()

        # Execute a query to retrieve table names
        query = f"SELECT table_name FROM INFORMATION_SCHEMA.TABLES WHERE table_schema = '{database}'"
        cursor.execute(query)

        # Fetch all the rows
        tables = cursor.fetchall()

        # Print table names
        # for (table_name,) in tables:
        #     print(table_name)

    except mysql.connector.Error as error:
        print(f"Error: {error}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            return tables
        

In [8]:
def table_to_dataframe(host, user, password, database, table_name):
    try:
        # Establish a database connection
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        # Use backticks around the table name in the query
        query = f"SELECT * FROM `{table_name}`"

        # Use pandas to read the query into a DataFrame
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            df = pd.read_sql(query, conn)

        return df

    except mysql.connector.Error as error:
        print(f"Error: {error}")
        return None
    finally:
        if conn.is_connected():
            conn.close()
            


In [9]:
databaseA = 'a20240112'
password = '93150lbm!!'

tables = get_table_names('127.0.0.1', 'root', password, databaseA)

tables = [code for (code,) in tables]
print(tables)


['000440', '005290', '009620', '009730', '013990', '021080', '024060', '025320', '027360', '027830', '029480', '033540', '036540', '036630', '038870', '041020', '041190', '041460', '047310', '048770', '049080', '050890', '053050', '056080', '057680', '058610', '059100', '060250', '060280', '060310', '064260', '065350', '065450', '066790', '068760', '072950', '080220', '080530', '085670', '086960', '088800', '090150', '090360', '090710', '094170', '094480', '096630', '098460', '099440', '104480', '108490', '115160', '117730', '129890', '136480', '142210', '203650', '205100', '205470', '207760', '208340', '214270', '214680', '215100', '218150', '227100', '228760', '232140', '241820', '253590', '254120', '264850', '270520', '270660', '277070', '290690', '291230', '293580', '299900', '306620', '307930', '317830', '319400', '321260', '321370', '348340', '356680', '380540', '382800', '383930', '388790', '391710', '403870', '417200', '419050', '419270', '450520', '452190', '452300', '900250']

In [71]:
windowsize = 30
windwosize_2 = 5

code = tables[6]

df = table_to_dataframe('127.0.0.1', 'root', password, databaseA, code)

df['체결시간'] = pd.to_datetime(df['체결시간'])
df['현재가'] = pd.to_numeric(df['현재가'].str.replace('+','').str.replace('-',''), errors='coerce')
df['거래량'] = pd.to_numeric(df['거래량'], errors='coerce')
df['거래대금'] = df['현재가'] * df['거래량']
df = df.sort_values(by='체결시간', ignore_index=True)
df['time_diff_seconds'] = df['체결시간'].diff().dt.total_seconds()
df['time_diff_mean'] = df['time_diff_seconds'].rolling(window=windowsize).mean()
df['exp_minus'] = np.exp(-df['time_diff_seconds'])
df['signal'] = df['exp_minus'] * df['거래대금']
df['signal_mean5'] = df['signal'].rolling(window=windwosize_2).mean()

# df = df[['체결시간','time_diff_seconds','거래대금','signal_mean5']]

# # df.to_csv(f'{databaseA}_{code}.csv', index=False)
# df.to_csv(f'{databaseA}_{code}.csv', index=False, encoding='utf-8-sig')

print(code)
df




024060


,id,체결시간,현재가,거래량,거래대금,time_diff_seconds,time_diff_mean,exp_minus,signal,signal_mean5
0,3,2024-01-12 09:41:32.600788,11880,-18,-213840,NaN,NaN,NaN,NaN,NaN
1,6,2024-01-12 09:41:32.757373,11880,-875,-10395000,0.156585,NaN,8.550588e-01,-8.888337e+06,NaN
2,7,2024-01-12 09:41:32.806488,11880,-83,-986040,0.049115,NaN,9.520716e-01,-9.387807e+05,NaN
3,12,2024-01-12 09:41:32.910937,11910,80,952800,0.104449,NaN,9.008207e-01,8.583020e+05,NaN
4,16,2024-01-12 09:41:33.006461,11890,-1,-11890,0.095524,NaN,9.088965e-01,-1.080678e+04,NaN
...,...,...,...,...,...,...,...,...,...,...
136276,136277,2024-01-12 15:42:20.263866,12740,200,2548000,2.299887,44.738774,1.002702e-01,2.554884e+05,222528.964437
136277,136278,2024-01-12 15:42:34.568669,12740,20,254800,14.304803,45.215601,6.130600e-07,1.562077e-01,51302.233514
136278,136279,2024-01-12 15:42:38.361683,12740,5,63700,3.793014,45.340317,2.252760e-02,1.435008e+03,51398.391389
136279,136280,2024-01-12 15:42:43.264068,12740,4,50960,4.902385,45.502485,7.428844e-03,3.785739e+02,51469.883534


In [72]:
df.set_index('체결시간', inplace=True)
df_resampled = df['현재가'].resample('T').last().ffill()
# Convert the resampled Series into a DataFrame
df_resampled = df_resampled.to_frame(name='현재가')

# Now you can add another column to it
# df_resampled['hello'] = 1

# df_resampled.head()  # Display the first few rows to check

df_resampled

,현재가
체결시간,
2024-01-12 09:41:00,11890.0
2024-01-12 09:42:00,11830.0
2024-01-12 09:43:00,11880.0
2024-01-12 09:44:00,12020.0
2024-01-12 09:45:00,12020.0
...,...
2024-01-12 15:38:00,12740.0
2024-01-12 15:39:00,12740.0
2024-01-12 15:40:00,12740.0


In [73]:
import pandas as pd

# Assuming 'df' is a DataFrame with a 'Close' column representing the closing prices of a stock

# Define the period for which you want to calculate RSI, typically 14
period_1 = 2
period_2 = 14
period_3 = 25

# Calculate the difference in price from previous step
delta = df_resampled['현재가'].diff(1)

# Get rid of the first NaN value
delta = delta.dropna()

# Make the positive gains (up) and negative gains (down) Series
up = delta.clip(lower=0)
down = -1 * delta.clip(upper=0)

# Calculate the average gain and average loss
avg_gain_1 = up.rolling(window=period_1).mean()
avg_loss_1 = down.rolling(window=period_1).mean()

avg_gain_2 = up.rolling(window=period_2).mean()
avg_loss_2 = down.rolling(window=period_2).mean()

avg_gain_3 = up.rolling(window=period_3).mean()
avg_loss_3 = down.rolling(window=period_3).mean()

# Calculate the relative strength (RS)
rs1 = avg_gain_1 / avg_loss_1
rs2 = avg_gain_2 / avg_loss_2
rs3 = avg_gain_3 / avg_loss_3

# Calculate the RSI
rsi1 = 100 - (100 / (1 + rs1))
rsi2 = 100 - (100 / (1 + rs2))
rsi3 = 100 - (100 / (1 + rs3))

# Attach the RSI to the original data
df_resampled['RSI_2'] = rsi1
df_resampled['RSI_21'] = rsi2
df_resampled['RSI_50'] = rsi3



# The 'RSI' column now contains the RSI values
df_resampled

,현재가,RSI_2,RSI_21,RSI_50
체결시간,,,,
2024-01-12 09:41:00,11890.0,NaN,NaN,NaN
2024-01-12 09:42:00,11830.0,NaN,NaN,NaN
2024-01-12 09:43:00,11880.0,45.454545,NaN,NaN
2024-01-12 09:44:00,12020.0,100.000000,NaN,NaN
2024-01-12 09:45:00,12020.0,100.000000,NaN,NaN
...,...,...,...,...
2024-01-12 15:38:00,12740.0,NaN,100.0,69.863014
2024-01-12 15:39:00,12740.0,NaN,100.0,62.068966
2024-01-12 15:40:00,12740.0,NaN,100.0,61.403509


In [75]:
df_resampled.to_csv(f'{databaseA}_{code}.csv', index=True, encoding='utf-8-sig')
